# 포켓몬 분류 모델

< 모듈 로딩 및 데이터 불러오기 > <hr>

In [1]:
# [1-1] 모듈 로딩
import cv2                           # 컴퓨터비젼 모듈
import numpy as np                   # 이미지 데이터 저장 모듈
import matplotlib.pyplot as plt      # 시각화 모듈
import os                            # 파일, 폴더, 경로 관련 모듈
import pandas as pd
from function import *               # score 값 등 한번에 시각화하는 함수 호출

In [2]:
# [1-2] 데이터 불러오기
pokemon_CSV = './4th_color.csv'

In [3]:
# [1-3] 이미지 데이터 로딩
pokemon_df = pd.read_csv(pokemon_CSV, header=None)  # header = None 지정하기!
pokemon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44840 entries, 0 to 44839
Columns: 14701 entries, 0 to 14700
dtypes: int64(14701)
memory usage: 4.9 GB


In [4]:
# header = None 지정하기!
display(pokemon_df.head(5))

,0,1,2,3,4,5,6,7,8,9,...,14691,14692,14693,14694,14695,14696,14697,14698,14699,14700
0,387,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,387,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,387,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,387,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,387,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


< 데이터 전처리 > <hr>

In [5]:
# [2-1] 피쳐 타겟 분리 
from sklearn.preprocessing import LabelEncoder

featureDF = pokemon_df[pokemon_df.columns[1:]]  # 피쳐 : 2차원
targetSR = pokemon_df[pokemon_df.columns[0]]    # 타겟 : 1차원

# 숫자 맞는지, 차원 맞는지 확인해주기
print(f'featureDF : {featureDF.shape}, targetSR : {targetSR.shape}')

featureDF : (44840, 14700), targetSR : (44840,)


In [6]:
# int64면 데이터 효율성이 떨어짐
print(featureDF.dtypes)

1        int64
2        int64
3        int64
4        int64
5        int64
         ...  
14696    int64
14697    int64
14698    int64
14699    int64
14700    int64
Length: 14700, dtype: object


In [7]:
print(featureDF.values)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [8]:
# [2-2] 피쳐 전처리 : 데이터 타입 확인, 값의 범위 맞추기

# 데이터 타입 변경 int64 -> uint8
featureDF = featureDF.astype('uint8')
featureDF.values

# 정규화 
# 0 ~ 255 -> 0.0 ~ 1.0 정규화 : 효율 증대
featureDF = featureDF/255.  # 정규화 과정에서 데이터프레임이나 배열의 모든 값이 동일한 타입으로 처리되도록 보장

featureDF

,1,2,3,4,5,6,7,8,9,10,...,14691,14692,14693,14694,14695,14696,14697,14698,14699,14700
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44835,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
44836,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
44837,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
44838,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [9]:
# [2-3] 타겟 전처리 :  str/object 타입의 경우 수치화 인코딩 
lencoder = LabelEncoder()
lencoder.fit(targetSR)
targetSR = lencoder.transform(targetSR)

# 확인 
print(lencoder.classes_)

[387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404
 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422
 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440
 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458
 459 460 461 462]


In [10]:
# [2-4] 학습용, 테스트용 데이터셋 분리
from sklearn.model_selection import train_test_split

# 분류 : stratify=targetSR
x_train ,x_test, y_train, y_test = train_test_split(featureDF, targetSR,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=targetSR)

In [11]:
print(f'x_train : {x_train.shape}   y_train : {y_train.shape}')
print(f'x_test : {x_test.shape}   y_test : {y_test.shape}')

x_train : (35872, 14700)   y_train : (35872,)
x_test : (8968, 14700)   y_test : (8968,)


< 학습 진행 > 

In [ ]:
# [3-1] 최적 알고리즘 찾기 
from sklearn.utils.discovery import *
from sklearn.metrics import *
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier

# all_estimators 함수 : 추정기 이름과 해당 클래스 튜플 리스트 반환
# {“classifier”, “regressor”, “cluster”, “transformer”} 
rets = all_estimators(type_filter='classifier')
classifiers = [
    ("XGBClassifier", XGBClassifier)]

# 아래 목록에서 돌릴 모델 하나씩 추가 
# ("LogisticRegression", LogisticRegression)
# ("RandomForest", RandomForestClassifier),
# ("GradientBoosting", GradientBoostingClassifier),
# ("AdaBoost", AdaBoostClassifier),
# ("KNeighbors", KNeighborsClassifier),
# ("SVC", SVC),  # SVM 포함
# ("DecisionTree", DecisionTreeClassifier),
# ("GaussianNB", GaussianNB),
# ("ExtraTrees", ExtraTreesClassifier),
# ("XGBClassifier", XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [ ]:
# 모델로 train score / test score 추출
resultList=[]
for name, estimator_ in classifiers:
    try:
        # 모델에 따라 필수 파라미터 지정
        model=estimator_()
        if 'Logistic' in name or 'SGD' in name or 'MLP' in name:
            model.set_params(max_iter=1000)
        if 'SV' in name:
            model.set_params(max_iter=5000, dual='auto')   
 
        model.fit(x_train, y_train)

        trainScore= model.score(x_train, y_train)
        testScore = model.score(x_test, y_test)

        resultList.append((name, round(trainScore, 3), round(testScore, 3)))
    
    # 모든 예외를 처리 
    except Exception:
        pass

In [ ]:
# 리스트의 2번 인덱스의 정보를 내림차순으로 정렬해서 표시 
sorted(resultList, key=lambda x : x[2], reverse=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# [3-2] 랜덤 포레스트 알고리즘 선정 
model = RandomForestClassifier(random_state=1)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=1)

In [16]:
# 혹시 모르니 예측값 반드시 확인
pre_y = model.predict(x_train)
print(pre_y)

[ 6 61  9 ... 33 28 36]


In [ ]:
# train
classify_train(x_train, y_train, model)

,정확도,정밀도,재현율,F1 스코어,오차
Train,1.0,1.0,1.0,1.0,0.272


In [ ]:
# test
classify_test(x_test, y_test, model)

,정확도,정밀도,재현율,F1 스코어,오차
Test,0.828613,0.835863,0.828613,0.829248,1.713


< 모델 저장 > <hr>

In [ ]:
# [4-1] 모델 저장 
import joblib 

MODEL_FILE = './model.pkl'

joblib.dump(model, MODEL_FILE)

['./model.pkl']

< 예측 > <hr>

In [ ]:
# [5-1] 예측 

# 함수 정의
IMG_DIR = './'
ROW, COL = 70, 70
FILE_CSV = './predict.csv'


def add_dataset(csv_file, label, im_data):
    try:
        # 이미지 데이터를 1차원 배열로 변환
        data = im_data.reshape(-1)
        # 문자열로 변환
        sdata = map(str, data)
        # CSV 파일에 라벨과 이미지 데이터 쓰기
        csv_file.write(f"{label}," + ','.join(sdata) + '\n')
    except Exception as e:
        print(f"add_dataset 함수 오류: {e}")

def preprocessing(filename, csv_file):
    try:
        # 컬러 이미지 읽기
        color_img = cv2.imread(IMG_DIR + filename, cv2.IMREAD_COLOR)
        if color_img is None:
            print(f"이미지를 읽을 수 없습니다: {filename}")
            return
        
        # 리사이즈
        color_img = cv2.resize(color_img, (ROW, COL))
        
        # 이미지를 파일로 저장
        if not cv2.imwrite(filename, color_img):
            print(f"{filename} - 저장 실패!")
        
        # 데이터 추가
        label = 390  
        add_dataset(csv_file, label, color_img)
    except Exception as e:
        print(f"preprocessing 함수 오류: {e}")

# CSV 파일 생성 및 데이터 추가
try:
    with open(FILE_CSV, mode="a", encoding="utf-8") as csv_file:
        preprocessing('predict1.png', csv_file)  # 예제 파일 추가
except Exception as e:
    print(f"CSV 파일 처리 오류: {e}")


In [ ]:
# [5-2] 예측 데이터 전처리까지 해서 저장 
predict_CSV = './predict.csv'

predict_df = pd.read_csv(predict_CSV, header=None)  

featureDF2 = predict_df[predict_df.columns[1:]]  # 피쳐 : 2차원
targetSR2 = predict_df[predict_df.columns[0]]    # 타겟 : 1차원


# 데이터 타입 변경 int64 -> uint8
featureDF2 = featureDF2.astype('uint8')
featureDF2.values

# 정규화 
# 0 ~ 255 -> 0.0 ~ 1.0 정규화 : 효율 증대
featureDF2 = featureDF2/255. 

new_image = featureDF2
 

prediction = model.predict(new_image)

print(prediction)

[3]


< 교차 검증 > <hr>

In [22]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [23]:
# 하이퍼파라미터 dict 생성
param = {'max_depth' : [i for i in range(2,9)], 'criterion' : ['gini', 'entropy', 'log_loss'],
         'min_samples_split': [10, 20],  # 분할 최소 샘플 수
        'min_samples_leaf': [5, 10],  # 리프 노드 최소 샘플 수
        'class_weight': [None, 'balanced']} # 데이터 불균형 처리


# 모델 인스턴스 생성
t_model = RandomForestClassifier(random_state=42)


# 교차검증과 튜닝 진행 인스턴스 생성
random_search = GridSearchCV(t_model, param, scoring=('accuracy'), refit='True')

In [24]:
random_search.fit(x_train, y_train)

KeyboardInterrupt: 

In [ ]:
# 가장 좋은 성능의 하이퍼파라미터 조합
params_df = pd.DataFrame([random_search.best_params_])
params_df

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
# 가장 좋은 모델 
best_model = random_search.best_estimator_

In [ ]:
classify_train(x_train, y_train, best_model)

In [ ]:
classify_test(x_test, y_test, best_model)